# VAE(変分オートエンコーダー)

## VAEとは？
VAE(変分オートエンコーダ)の説明の前にオートエンコーダーとはなにか説明していきます  
  
### ・オートエンコーダー
オートエンコーダーとは、教師なし学習の一つでデータを表現する特徴を得るためのニューラルネットワークである。  
  
オートエンコーダーの目標はデコーダーからもとの入力データを再現できるよう特徴表現を学習することです。  
  
主な活用事例としてデータのノイズ除去やクラスタリング、そしてデータ生成など  
  
幅広く活用されています。

### ・変分オートエンコーダー
  
VAEはオートエンコーダーの拡張版で、データ生成に焦点を当てたモデルです。  
  
４章で話したようにデータの形は正規分布一個で表せるほど単純なものではありません。  
  
そこでGMMを学んだわけですが、  
  
今回はGMMよりも表現力を向上させるため学習データに合わせて柔軟に形が決定できるものを目指していきます。